In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    !pip install -q google-genai google-auth python-dotenv
    from google.colab import auth
    auth.authenticate_user()
    try:
        PROJECT_ID = input("Enter your Google Cloud Project ID (press Enter to use default ADC): ").strip()
    except Exception:
        PROJECT_ID = ""
    if PROJECT_ID:
        import os
        os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

import os
import google.auth
from google import genai
from google.genai import types

creds, project = google.auth.default()
project = os.environ.get("GOOGLE_CLOUD_PROJECT", project)
client = genai.Client(vertexai=True, project=project, location="us-central1")
print(f"Using project: {project}")

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kgweber-cwru/coding-with-ai-wn26/blob/main/series-2-coding-llms/week-2-conversations/assignment.ipynb)

# Week 2 Assignment: Build a Conversational Application

## Objective
Create a conversational application that maintains context across multiple turns and serves a specific purpose in your domain.

## Requirements
1. Use the Conversation class (or extend it)
2. Include a well-crafted system message
3. Demonstrate at least 5 conversation turns
4. Show how context is maintained across turns
5. Track token usage and estimate costs

## Ideas
- **Research advisor**: Helps refine research questions through dialogue
- **Study buddy**: Explains concepts and answers follow-up questions
- **Interview bot**: Conducts structured interviews for qualitative research
- **Writing coach**: Provides feedback and suggestions iteratively
- **Clinical decision support**: Helps work through diagnostic reasoning
- **Data analysis helper**: Assists with data interpretation questions
- **Grant writing assistant**: Helps develop and refine grant proposals
- **Literature review helper**: Discusses papers and synthesizes themes

## Setup

In [ ]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types
import google.auth

load_dotenv()
creds, project = google.auth.default()
client = genai.Client(vertexai=True, project=project, location="us-central1")

print("✓ Setup complete!")

## Copy the Conversation Class
Use the class from the concepts notebook (or extend it!):

In [ ]:
class Conversation:
    """A simple conversation manager"""
    
    def __init__(self, system_message="You are a helpful assistant.", model="gemini-1.5-flash"):
        self.system_message = system_message
        self.messages = [] # History of user/model turns
        self.model = model
        self.total_tokens = 0
    
    def add_user_message(self, content):
        """Add a user message to the conversation"""
        self.messages.append({"role": "user", "content": content})
    
    def get_response(self, temperature=0.7, max_tokens=None):
        """Get assistant response and add to history"""
        # Convert internal message format to Vertex AI Content objects
        content_list = [
            types.Content(role=m["role"], parts=[types.Part.from_text(m["content"])]) 
            for m in self.messages
        ]

        config = types.GenerateContentConfig(
            system_instruction=self.system_message,
            temperature=temperature,
            max_output_tokens=max_tokens
        )
        
        response = client.models.generate_content(
            model=self.model,
            contents=content_list,
            config=config
        )
        
        model_message = response.text
        self.messages.append({"role": "model", "content": model_message})
        
        if response.usage_metadata:
            self.total_tokens += response.usage_metadata.total_token_count
        
        return model_message
    
    def chat(self, user_message, temperature=0.7, max_tokens=None):
        """Convenience method: add user message and get response"""
        self.add_user_message(user_message)
        return self.get_response(temperature, max_tokens)
    
    def display_history(self):
        """Display the conversation history"""
        print(f"SYSTEM: {self.system_message}")
        print("-" * 50)
        for msg in self.messages:
            role = msg["role"].upper()
            content = msg["content"]
            print(f"{role}: {content}")
            print("-" * 50)
    
    def get_token_count(self):
        """Get total tokens used"""
        return self.total_tokens

## Your Application

### Step 1: Describe Your Application

**YOUR DESCRIPTION HERE**

I'm building: [describe your conversational application]

Purpose: [what problem does it solve?]

Target users: [who would use this?]

How it works: [brief explanation of the conversation flow]

### Step 2: Design Your System Message

Carefully craft a system message that defines:
- The assistant's role
- Its expertise area
- How it should respond
- Any specific behaviors or constraints

In [ ]:
# YOUR SYSTEM MESSAGE HERE
system_message = """
YOUR WELL-CRAFTED SYSTEM MESSAGE
"""

# Create your conversation
my_assistant = Conversation(system_message=system_message)

print("✓ Assistant created!")

### Step 3: Demonstrate Your Application

Show a realistic conversation with at least 5 turns. Each turn should:
- Build on previous context
- Demonstrate the assistant's capabilities
- Show value to the user

In [ ]:
# Turn 1
print("USER: [your message]")
print("\nASSISTANT:", my_assistant.chat("YOUR MESSAGE HERE"))
print("\n" + "="*70 + "\n")

In [ ]:
# Turn 2
print("USER: [your follow-up message]")
print("\nASSISTANT:", my_assistant.chat("YOUR FOLLOW-UP HERE"))
print("\n" + "="*70 + "\n")

In [ ]:
# Turn 3
# YOUR CODE HERE


In [ ]:
# Turn 4
# YOUR CODE HERE


In [ ]:
# Turn 5
# YOUR CODE HERE


### Step 4: Analyze the Conversation

In [ ]:
# Display full conversation history
print("FULL CONVERSATION HISTORY:")
print("="*70)
my_assistant.display_history()

In [ ]:
# Token usage and cost
total_tokens = my_assistant.get_token_count()
# Rough estimate for Gemini 1.5 Flash (blended rate, approx $0.15 per 1M tokens)
estimated_cost = (total_tokens / 1_000_000) * 0.15  

print(f"Total tokens used: {total_tokens}")
print(f"Estimated cost: ${estimated_cost:.6f}")
print(f"Average tokens per turn: {total_tokens / 5:.1f}")

## Reflection Questions

### 1. How did the system message shape the conversation?

**YOUR ANSWER HERE**

### 2. Show an example where context from an earlier turn influenced a later response

**YOUR ANSWER HERE**

Example: In turn [X], I mentioned [Y], and in turn [Z], the assistant referred back to this by [describe how]...

### 3. What would happen if this conversation continued for 20 more turns?

**YOUR ANSWER HERE**

Consider: token costs, context window limits, potential drift from original purpose...

### 4. How could you improve this application?

**YOUR ANSWER HERE**

## Bonus Challenges (Optional)

### Challenge 1: Extend the Conversation Class
Add features like:
- Saving/loading conversations to file
- Automatic summarization when history gets long
- Conversation branching
- Export to different formats

In [ ]:
# BONUS CODE HERE


### Challenge 2: Compare Temperature Settings
Run the same conversation with different temperatures (0.0, 0.7, 1.2) and compare responses.

In [ ]:
# BONUS CODE HERE


### Challenge 3: Build a Multi-Persona System
Create a system that can switch between different expert personas within a single conversation.

In [ ]:
# BONUS CODE HERE


## Submission Checklist

- [ ] Described your application and its purpose
- [ ] Crafted a thoughtful system message
- [ ] Demonstrated at least 5 conversation turns
- [ ] Showed how context is maintained
- [ ] Analyzed token usage and costs
- [ ] Answered all reflection questions
- [ ] Tested with realistic examples
- [ ] Saved your notebook!

## Next Week Preview

Next week: **Programmatic Prompt Engineering**
- Dynamic prompt generation
- Template systems
- Few-shot learning
- Output parsing and validation

Start thinking about tasks where you need to generate similar prompts with different data!